# BM25 Search Engine - Vietnamese Football News

## 📋 Mục đích
Notebook này implement BM25 ranking algorithm để tìm kiếm và xếp hạng documents từ dữ liệu bóng đá Việt Nam.

## 🎯 Chức năng chính:
1. Load và preprocess documents từ JSON files
2. Build BM25 index với Vietnamese text processing
3. Query search với ranking scores
4. Hiển thị kết quả chi tiết (title, snippet, score, document info)

## 1. Import Libraries

In [1]:
import os
import json
import re
from collections import Counter
from tqdm import tqdm
import pandas as pd

# BM25
try:
    from rank_bm25 import BM25Okapi
    print("✓ rank_bm25 đã được cài đặt")
except ImportError:
    print("✗ Chưa có rank_bm25. Cài đặt: pip install rank-bm25")

# PyVi for Vietnamese tokenization
try:
    from pyvi import ViTokenizer
    PYVI_AVAILABLE = True
    print("✓ PyVi đã được cài đặt")
except ImportError:
    print("✗ PyVi chưa cài đặt. Cài đặt bằng: pip install pyvi")
    PYVI_AVAILABLE = False

✓ rank_bm25 đã được cài đặt
✓ PyVi đã được cài đặt


## 2. Vietnamese Text Processor Class

In [2]:
class VietnameseTextProcessor:
    """Xử lý văn bản tiếng Việt cho BM25"""
    
    def __init__(self):
        # Vietnamese stopwords
        self.stop_words = set([
            'và', 'của', 'trong', 'với', 'là', 'có', 'được', 'cho', 'từ', 'một', 'các',
            'để', 'không', 'sẽ', 'đã', 'về', 'hay', 'theo', 'như', 'cũng', 'này', 'đó',
            'khi', 'những', 'tại', 'sau', 'bị', 'giữa', 'trên', 'dưới', 'ngoài',
            'thì', 'nhưng', 'mà', 'hoặc', 'nếu', 'vì', 'do', 'nên', 'rồi', 'còn', 'đều',
            'chỉ', 'việc', 'người', 'lại', 'đây', 'đấy', 'ở', 'ra', 'vào', 'lên', 'xuống'
        ])
    
    def clean_text(self, text):
        """Làm sạch và chuẩn hóa text tiếng Việt"""
        if not text:
            return ""
        
        # Loại bỏ ký tự đặc biệt, giữ lại tiếng Việt
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\sàáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđĐ]', ' ', text)
        text = text.lower()
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    def tokenize(self, text):
        """Tách từ tiếng Việt"""
        if PYVI_AVAILABLE:
            try:
                return ViTokenizer.tokenize(text).split()
            except:
                pass
        return text.split()
    
    def remove_stopwords(self, tokens):
        """Loại bỏ stopwords"""
        return [token for token in tokens if token not in self.stop_words and len(token) > 1]
    
    def preprocess(self, text):
        """Pipeline xử lý hoàn chỉnh"""
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        filtered = self.remove_stopwords(tokens)
        return filtered

print("✓ VietnameseTextProcessor class đã được định nghĩa")

✓ VietnameseTextProcessor class đã được định nghĩa


## 3. BM25 Search Engine Class

In [ ]:
class BM25SearchEngine:
    """BM25 Search Engine cho Vietnamese Football News"""
    
    def __init__(self):
        self.processor = VietnameseTextProcessor()
        self.bm25 = None
        self.documents = []
        self.corpus_tokens = []
        
    def load_data(self, json_files=None):
        """Load documents từ JSON files"""
        if json_files is None:
            json_files = [
                "vnexpressT_bongda_part1.json",
                "vnexpressT_bongda_part2.json",
                "vnexpressT_bongda_part3.json",
                "vnexpressT_bongda_part4.json"
            ]
        
        print("📥 Loading documents from JSON files...")
        for file_path in json_files:
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as f:
                    try:
                        data = json.load(f)
                        if isinstance(data, list):
                            self.documents.extend(data)
                        print(f"  ✓ Loaded {file_path}: {len(data)} documents")
                    except Exception as e:
                        print(f"  ✗ Error reading {file_path}: {e}")
            else:
                print(f"  ✗ File not found: {file_path}")
        
        print(f"\n✓ Total documents loaded: {len(self.documents)}")
        return self.documents
    
    def build_index(self):
        """Build BM25 index từ documents"""
        if not self.documents:
            print("✗ No documents to index. Load data first!")
            return
        
        print("\n Building BM25 index...")
        self.corpus_tokens = []
        
        for doc in tqdm(self.documents, desc="Processing documents"):
            # Combine title and content
            title = doc.get('title', '')
            content = doc.get('content', '')
            full_text = f"{title} {content}"
            
            # Tokenize and preprocess
            tokens = self.processor.preprocess(full_text)
            self.corpus_tokens.append(tokens)
        
        # Create BM25 index
        self.bm25 = BM25Okapi(self.corpus_tokens)
        print(f"✓ BM25 index built with {len(self.corpus_tokens)} documents\n")
    
    def search(self, query, top_k=10):
        """Search với BM25 ranking"""
        if self.bm25 is None:
            print("✗ BM25 index not built. Run build_index() first!")
            return []
        
        # Preprocess query
        query_tokens = self.processor.preprocess(query)
        
        # Get BM25 scores
        scores = self.bm25.get_scores(query_tokens)
        
        # Get top-k results
        top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        
        results = []
        for idx in top_indices:
            results.append({
                'rank': len(results) + 1,
                'doc_index': idx,
                'score': scores[idx],
                'document': self.documents[idx]
            })
        
        return results
    
    def display_results(self, results, show_content=True, max_content_length=300):
        """Hiển thị kết quả tìm kiếm"""
        if not results:
            print("❌ No results found!")
            return
        
        print(f"\n{'='*100}")
        print(f"FOUND {len(results)} RESULTS")
        print(f"{'='*100}\n")
        
        for result in results:
            rank = result['rank']
            score = result['score']
            doc = result['document']
            
            title = doc.get('title', 'No title')
            url = doc.get('url', 'No URL')
            date = doc.get('date', 'No date')
            author = doc.get('author', 'Unknown')
            content = doc.get('content', '')
            
            # Create content snippet
            if content and len(content) > max_content_length:
                snippet = content[:max_content_length] + "..."
            else:
                snippet = content
            
            print(f"[{rank}]  SCORE: {score:.4f}")
            print(f" Title: {title}")
            print(f" Date: {date} | ✍️ Author: {author}")
            print(f" URL: {url}")
            
            if show_content:
                print(f" Content: {snippet}")
            
            print(f"{'-'*100}\n")
    
    def search_and_display(self, query, top_k=10, show_content=True, max_content_length=300):
        """Thực hiện search và hiển thị kết quả ngay"""
        print(f"\n SEARCHING FOR: \"{query}\"\n")
        results = self.search(query, top_k)
        self.display_results(results, show_content, max_content_length)
        return results
    
    def get_statistics(self):
        """Thống kê về corpus"""
        if not self.documents:
            print("No documents loaded!")
            return
        
        total_docs = len(self.documents)
        avg_tokens = sum(len(tokens) for tokens in self.corpus_tokens) / total_docs if self.corpus_tokens else 0
        
        print(f"\n CORPUS STATISTICS")
        print(f"{'='*50}")
        print(f"Total documents: {total_docs}")
        print(f"Average tokens per document: {avg_tokens:.1f}")
        
        if self.corpus_tokens:
            all_tokens = [token for tokens in self.corpus_tokens for token in tokens]
            vocab_size = len(set(all_tokens))
            print(f"Vocabulary size: {vocab_size}")
            
            # Top 10 most common words
            counter = Counter(all_tokens)
            print(f"\nTop 10 most common words:")
            for word, count in counter.most_common(10):
                print(f"  {word}: {count}")

print("✓ BM25SearchEngine class đã được định nghĩa")

✓ BM25SearchEngine class đã được định nghĩa


## 4. Initialize & Load Data

In [4]:
# Khởi tạo BM25 Search Engine
bm25_engine = BM25SearchEngine()

# Load data từ JSON files
documents = bm25_engine.load_data()

print(f"\n✅ Loaded {len(documents)} documents successfully!")

📥 Loading documents from JSON files...
  ✓ Loaded vnexpressT_bongda_part1.json: 473 documents


  ✓ Loaded vnexpressT_bongda_part2.json: 488 documents
  ✓ Loaded vnexpressT_bongda_part3.json: 487 documents
  ✓ Loaded vnexpressT_bongda_part4.json: 308 documents

✓ Total documents loaded: 1756

✅ Loaded 1756 documents successfully!


## 5. Build BM25 Index

In [5]:
# Build BM25 index
bm25_engine.build_index()

# Show statistics
bm25_engine.get_statistics()


🏗️ Building BM25 index...


Processing documents: 100%|██████████| 1756/1756 [01:03<00:00, 27.76it/s]


✓ BM25 index built with 1756 documents


📊 CORPUS STATISTICS
Total documents: 1756
Average tokens per document: 424.8
Vocabulary size: 15607

Top 10 most common words:
  nam: 8293
  đội: 7750
  trận: 7581
  hai: 6792
  việt: 6762
  cầu_thủ: 6677
  bóng: 5719
  hlv: 5612
  league: 5192
  năm: 5190


## 6. Search Queries

### 6.1. Example Search Query 1: "Bóng đá Việt Nam"

In [6]:
# Search query về bóng đá Việt Nam
results = bm25_engine.search_and_display(
    query="bóng đá việt nam",
    top_k=5,
    show_content=True,
    max_content_length=200
)


🔍 SEARCHING FOR: "bóng đá việt nam"


FOUND 5 RESULTS

[1] 🏆 SCORE: 3.8253
📰 Title: Honda tặng xe cho hai đội tuyển bóng đá Việt Nam
📅 Date: Thứ năm, 12/12/2019, 15:45 (GMT+7) | ✍️ Author: Tuấn Vũ
🔗 URL: https://vnexpress.net/honda-tang-xe-cho-hai-doi-tuyen-bong-da-viet-nam-4026276.html
📄 Content: Tối 11/12 vừa qua, Honda Việt Nam đã trao phần thưởng mừng chiến thắng của đội tuyển bóng đá nữ quốc gia và đội tuyển bóng đá nam U22 Việt Nam, tại Văn phòng Chính phủ. Thủ tướng Nguyễn Xuân Phúc cùng...
----------------------------------------------------------------------------------------------------

[2] 🏆 SCORE: 3.6570
📰 Title: Báo chí Hàn Quốc: ‘Đây là bóng đá Việt Nam sao?’
📅 Date: Thứ ba, 23/1/2018, 19:32 (GMT+7) | ✍️ Author: Xuân Bình
🔗 URL: https://vnexpress.net/bao-chi-han-quoc-day-la-bong-da-viet-nam-sao-3702883.html
📄 Content: Với tiêu đề: “Đây là bóng đá Việt Nam sao? Đúng rồi đấy”, tờ DongA sốc với chiến thắng 4-3 của Việt Nam trong loạt đá luân lưu, sau khi hòa 2-2 trong 120 

### 6.2. Example Search Query 2: "Quang Hải cầu thủ"

In [7]:
# Search query về cầu thủ Quang Hải
results = bm25_engine.search_and_display(
    query="quang hải cầu thủ",
    top_k=5,
    show_content=True,
    max_content_length=200
)


🔍 SEARCHING FOR: "quang hải cầu thủ"


FOUND 5 RESULTS

[1] 🏆 SCORE: 5.4361
📰 Title: Quang Hải: ‘Mong đến ngày có cầu thủ Việt Nam đá ở Ngoại hạng Anh’
📅 Date: Chủ nhật, 16/9/2018, 16:42 (GMT+7) | ✍️ Author: Lâm Thỏa
🔗 URL: https://vnexpress.net/quang-hai-mong-den-ngay-co-cau-thu-viet-nam-da-o-ngoai-hang-anh-3809841.html
📄 Content: Quang Hải cùng cựu hậu vệ Joleon Lescott của Man City và Cup vô địch Ngoại hạng Anh, League Cup. “Tôi may mắn là cầu thủ Việt Nam đầu tiên được chạm tay vào chiếc Cup vô địch Ngoại hạng Anh và League ...
----------------------------------------------------------------------------------------------------

[2] 🏆 SCORE: 5.2683
📰 Title: Khi Lee Nguyễn vẫn là ngôi sao V-League
📅 Date: Thứ ba, 23/3/2021, 12:33 (GMT+7) | ✍️ Author: Song Việt
🔗 URL: https://vnexpress.net/khi-lee-nguyen-van-la-ngoi-sao-v-league-4252605.html
📄 Content: Nếu cần phải tìm một ai đó làm thước đo về đẳng cấp của cầu thủ Việt Nam, thì so sánh với Lee Nguyễn có lẽ sẽ mang đến sự chính xác t

### 6.3. Example Search Query 3: "V-League 2024"

In [8]:
# Search query về V-League
results = bm25_engine.search_and_display(
    query="v-league 2024",
    top_k=5,
    show_content=True,
    max_content_length=200
)


🔍 SEARCHING FOR: "v-league 2024"


FOUND 5 RESULTS

[1] 🏆 SCORE: 4.4433
📰 Title: V-League 2024-2025 chỉ tạm dừng vì FIFA days và AFF Cup
📅 Date: Thứ năm, 11/4/2024, 06:07 (GMT+7) | ✍️ Author: Hiếu Lương
🔗 URL: https://vnexpress.net/v-league-2024-2025-chi-tam-dung-vi-fifa-days-va-aff-cup-4732862.html
📄 Content: Đội tuyển Việt Nam sẽ có năm đợt FIFA days, gồm từ ngày 2/9 đến 10/9, ngày 7/10 đến 15/10, ngày 11/11 đến 19/11, ngày 17/3/2025 đến 25/3/2025 và ngày 2/6/2025 đến 10/6/2025. Trong khi đó, AFF Cup 2024...
----------------------------------------------------------------------------------------------------

[2] 🏆 SCORE: 4.4364
📰 Title: V-League ra tối hậu thư cho Quảng Nam
📅 Date: Thứ tư, 23/7/2025, 14:56 (GMT+7) | ✍️ Author: Hiếu Lương
🔗 URL: https://vnexpress.net/v-league-ra-toi-hau-thu-cho-quang-nam-4917967.html
📄 Content: Liên tiếp trong hai ngày 22/7 và 23/7, đơn vị tổ chức V-League là Công ty cổ phần bóng đá chuyên nghiệp Việt Nam (VPF) đã gửi hai công văn đến CLB Quảng Nam. 

### 6.4. Example Search Query 4: "HLV Park Hang-seo"

In [9]:
# Search query về HLV Park Hang-seo
results = bm25_engine.search_and_display(
    query="hlv park hang seo",
    top_k=5,
    show_content=True,
    max_content_length=200
)


🔍 SEARCHING FOR: "hlv park hang seo"


FOUND 5 RESULTS

[1] 🏆 SCORE: 11.5520
📰 Title: HLV Park Hang-seo nói gì với các cầu thủ Việt Nam sau trận thắng Iraq
📅 Date: Chủ nhật, 21/1/2018, 20:22 (GMT+7) | ✍️ Author: Lâm Thỏa
🔗 URL: https://vnexpress.net/hlv-park-hang-seo-noi-gi-voi-cac-cau-thu-viet-nam-sau-tran-thang-iraq-3701712.html
📄 Content: HLV Park Hang-seo cùng U23 Việt Nam viết chuyện cổ tích ở vòng chung kết U23 châu Á. Ảnh: Anh Khoa “Một suất ở bán kết đã thuộc về các bạn. Các bạn xứng đáng. Không có may mắn nào lập được kỳ tích đâu...
----------------------------------------------------------------------------------------------------

[2] 🏆 SCORE: 11.5289
📰 Title: Cầu thủ Việt Nam tìm lại nụ cười sau trận thua Iraq
📅 Date: Thứ tư, 9/1/2019, 16:51 (GMT+7) | ✍️ Author: Ảnh:Anh Khoa
🔗 URL: https://vnexpress.net/cau-thu-viet-nam-tim-lai-nu-cuoi-sau-tran-thua-iraq-3865973.html
📄 Content: Một ngày sau trận thua Iraq 2-3, các cầu thủ Việt Nam đã ra sân tập để chuẩn bị cho trận đấu tiế

### 6.5. Custom Search - Thử nghiệm với query của bạn

In [10]:
# Thử nghiệm với query tùy chỉnh
custom_query = "đội tuyển việt nam"  # Thay đổi query này theo ý bạn

results = bm25_engine.search_and_display(
    query=custom_query,
    top_k=10,  # Số lượng kết quả muốn hiển thị
    show_content=True,  # True để show content, False để chỉ show title
    max_content_length=300  # Độ dài tối đa của content snippet
)


🔍 SEARCHING FOR: "đội tuyển việt nam"


FOUND 10 RESULTS

[1] 🏆 SCORE: 4.0826
📰 Title: Honda tặng xe cho hai đội tuyển bóng đá Việt Nam
📅 Date: Thứ năm, 12/12/2019, 15:45 (GMT+7) | ✍️ Author: Tuấn Vũ
🔗 URL: https://vnexpress.net/honda-tang-xe-cho-hai-doi-tuyen-bong-da-viet-nam-4026276.html
📄 Content: Tối 11/12 vừa qua, Honda Việt Nam đã trao phần thưởng mừng chiến thắng của đội tuyển bóng đá nữ quốc gia và đội tuyển bóng đá nam U22 Việt Nam, tại Văn phòng Chính phủ. Thủ tướng Nguyễn Xuân Phúc cùng đại diện Honda Việt Nam trao tặng dàn xe Honda Lead cho đội tuyển bóng đá nữ. Theo đó, đội tuyển bó...
----------------------------------------------------------------------------------------------------

[2] 🏆 SCORE: 3.8101
📰 Title: HLV Park: 'Tôi muốn giúp bóng đá Việt Nam mạnh hơn'
📅 Date: Thứ ba, 5/11/2019, 18:00 (GMT+7) | ✍️ Author: Đức Đồng
🔗 URL: https://vnexpress.net/hlv-park-toi-muon-giup-bong-da-viet-nam-manh-hon-4007892.html
📄 Content: "Trước tiên, tôi xin cảm ơn tất cả người hâm 

## 7. Advanced: Compare Multiple Queries

In [11]:
# So sánh kết quả của nhiều queries
test_queries = [
    "công phượng bàn thắng",
    "asean cup 2024",
    "thủ môn đặng văn lâm",
    "hà nội fc vô địch"
]

print("\n" + "="*100)
print("COMPARING MULTIPLE QUERIES")
print("="*100)

for query in test_queries:
    results = bm25_engine.search(query, top_k=3)
    print(f"\n🔍 Query: \"{query}\"")
    print(f"Top 3 results:")
    for r in results:
        print(f"  [{r['rank']}] Score: {r['score']:.4f} - {r['document'].get('title', 'No title')[:80]}")
    print("-" * 100)


COMPARING MULTIPLE QUERIES

🔍 Query: "công phượng bàn thắng"
Top 3 results:
  [1] Score: 11.4124 - HAGL ghi 36% số bàn phạt đền ở V-League 2021
  [2] Score: 11.3702 - Vòng 6 V-League và sự khác biệt từ Công Phượng
  [3] Score: 10.6776 - Công Phượng rời tuyển Việt Nam
----------------------------------------------------------------------------------------------------

🔍 Query: "asean cup 2024"
Top 3 results:
  [1] Score: 4.7703 - Việt Nam thắng dễ trận ra quân vòng loại Asian Cup
  [2] Score: 4.7473 - 5 cầu thủ đáng xem nhất bảng B ASEAN Cup 2024
  [3] Score: 4.6883 - ASEAN Cup có thể đá vào giữa năm
----------------------------------------------------------------------------------------------------

🔍 Query: "thủ môn đặng văn lâm"
Top 3 results:
  [1] Score: 10.9399 - Đặng Văn Lâm làm đội trưởng ở V-League 2023
  [2] Score: 10.6954 - Đặng Văn Lâm: 'Gặp tuyển Việt Nam không dễ cho Nga'
  [3] Score: 10.5588 - Đặng Văn Lâm trở lại tuyển Việt Nam
------------------------------------------

## 8. Export Results to DataFrame

In [ ]:
# Export kết quả search thành DataFrame để dễ phân tích
def results_to_dataframe(results):
    """Convert search results to pandas DataFrame"""
    data = []
    for r in results:
        doc = r['document']
        data.append({
            'Rank': r['rank'],
            'Score': round(r['score'], 4),
            'Title': doc.get('title', 'No title'),
            'Date': doc.get('date', 'No date'),
            'Author': doc.get('author', 'Unknown'),
            'URL': doc.get('url', 'No URL')
        })
    return pd.DataFrame(data)

query = "bóng đá việt nam"
results = bm25_engine.search(query, top_k=10)
df_results = results_to_dataframe(results)

print(f"\n🔍 Query: \"{query}\"\n")
print(df_results.to_string(index=False))



🔍 Query: "bóng đá việt nam"

 Rank  Score                                                           Title                               Date               Author                                                                                              URL
    1 3.8253                Honda tặng xe cho hai đội tuyển bóng đá Việt Nam Thứ năm, 12/12/2019, 15:45 (GMT+7)              Tuấn Vũ              https://vnexpress.net/honda-tang-xe-cho-hai-doi-tuyen-bong-da-viet-nam-4026276.html
    2 3.6570                Báo chí Hàn Quốc: ‘Đây là bóng đá Việt Nam sao?’   Thứ ba, 23/1/2018, 19:32 (GMT+7)            Xuân Bình                  https://vnexpress.net/bao-chi-han-quoc-day-la-bong-da-viet-nam-sao-3702883.html
    3 3.6058         Nhật xuất bản truyện tranh đầu tiên về bóng đá Việt Nam   Thứ tư, 30/3/2022, 13:56 (GMT+7) Hồng Hạnh(TheoTTXVN)       https://vnexpress.net/nhat-xuat-ban-truyen-tranh-dau-tien-ve-bong-da-viet-nam-4445247.html
    4 3.5989             HLV Park: 'Tôi muốn giúp 

## 9. Performance Analysis

In [13]:
import time

# Đo thời gian search
test_queries = [
    "bóng đá việt nam",
    "quang hải",
    "v-league",
    "đội tuyển",
    "park hang seo"
]

print("\n⏱️ PERFORMANCE ANALYSIS")
print("="*70)

total_time = 0
for query in test_queries:
    start_time = time.time()
    results = bm25_engine.search(query, top_k=10)
    elapsed_time = time.time() - start_time
    total_time += elapsed_time
    
    print(f"Query: '{query:30s}' | Time: {elapsed_time*1000:.2f}ms | Results: {len(results)}")

avg_time = total_time / len(test_queries)
print("="*70)
print(f"Average search time: {avg_time*1000:.2f}ms")
print(f"Total queries tested: {len(test_queries)}")


⏱️ PERFORMANCE ANALYSIS
Query: 'bóng đá việt nam              ' | Time: 10.65ms | Results: 10
Query: 'quang hải                     ' | Time: 6.00ms | Results: 10
Query: 'v-league                      ' | Time: 5.00ms | Results: 10
Query: 'đội tuyển                     ' | Time: 4.22ms | Results: 10
Query: 'park hang seo                 ' | Time: 3.19ms | Results: 10
Average search time: 5.81ms
Total queries tested: 5


## 11. DEMO - Tìm Từ Khóa Trong Documents

### Cách dùng: Thay đổi `KEYWORD` và chạy cell!

In [ ]:
#  DEMO: TÌM TỪ KHÓA TRONG DOCUMENTS
# ========================================

#  NHẬP TỪ KHÓA BẠN MUỐN TÌM Ở ĐÂY:
KEYWORD = input("")  # Thay đổi từ khóa này và chạy lại cell

#  Số lượng kết quả muốn xem:
TOP_N = 5

# ========================================

print(f"\n{'='*100}")
print(f" Tìm kiếm từ khóa: '{KEYWORD}'")
print(f"{'='*100}\n")

keyword_lower = KEYWORD.lower()

# Tìm documents chứa keyword
found_docs = []
for idx, doc in enumerate(bm25_engine.documents):
    title = doc.get('title', '')
    content = doc.get('content', '')
    full_text = f"{title} {content}".lower()
    
    if keyword_lower in full_text:
        count = full_text.count(keyword_lower)
        found_docs.append({
            'index': idx,
            'count': count,
            'doc': doc
        })

# Sắp xếp theo số lần xuất hiện
found_docs.sort(key=lambda x: x['count'], reverse=True)

# Hiển thị kết quả
if found_docs:
    print(f"Tìm thấy '{KEYWORD}' trong {len(found_docs)} documents!\n")
    print(f"{'='*100}")
    print(f"TOP {min(TOP_N, len(found_docs))} DOCUMENTS CHỨA '{KEYWORD}'")
    print(f"{'='*100}\n")
    
    for i, item in enumerate(found_docs[:TOP_N], 1):
        doc = item['doc']
        count = item['count']
        
        print(f"[{i}] Xuất hiện: {count} lần")
        print(f"Title: {doc.get('title', 'No title')}")
        print(f"Date: {doc.get('date', 'No date')}")
        
        # Hiển thị đoạn text chứa từ khóa
        full_text = f"{doc.get('title', '')} {doc.get('content', '')}"
        pos = full_text.lower().find(keyword_lower)
        if pos != -1:
            start = max(0, pos - 80)
            end = min(len(full_text), pos + len(KEYWORD) + 80)
            snippet = full_text[start:end]
            if start > 0:
                snippet = "..." + snippet
            if end < len(full_text):
                snippet = snippet + "..."
            print(f"Snippet: {snippet}")
        
        print(f"{'-'*100}\n")
    
    # Tổng kết
    total = sum(item['count'] for item in found_docs)
    print(f"TỔNG KẾT:")
    print(f"  ✓ Tìm thấy trong {len(found_docs)} documents")
    print(f"  ✓ Tổng cộng xuất hiện: {total} lần")
    print(f"  ✓ Trung bình: {total/len(found_docs):.1f} lần/document")
else:
    print(f" Không tìm thấy '{KEYWORD}' trong documents!")

print(f"\n{'='*100}")


🔍 Tìm kiếm từ khóa: 'Mỹ Đình'

Tìm thấy 'Mỹ Đình' trong 157 documents!

TOP 5 DOCUMENTS CHỨA 'Mỹ Đình'

[1] Xuất hiện: 10 lần
Title: Thế khó của Thể Công với sân Mỹ Đình
Date: Thứ ba, 1/4/2025, 21:02 (GMT+7)
Snippet: Thế khó của Thể Công với sân Mỹ Đình Thể Công từng thuê sân Hàng Đẫy làm sân nhà từ năm 2019, cùng Hà Nội FC. Tới mù...
----------------------------------------------------------------------------------------------------

[2] Xuất hiện: 8 lần
Title: CĐV khuấy động trước trận Việt Nam - Indonesia
Date: Thứ ba, 26/3/2024, 17:56 (GMT+7)
Snippet: ...sao vàng... ... và chiêng, trống chở người hâm mộ đi trên các tuyến phố gần sân Mỹ Đình. Bất mãn sau những màn trình diễn gần đây của thầy trò Philippe Troussier, lượn...
----------------------------------------------------------------------------------------------------

[3] Xuất hiện: 7 lần
Title: Sân Mỹ Đình khó tổ chức trận Việt Nam - Indonesia
Date: Thứ sáu, 8/11/2024, 12:43 (GMT+7)
Snippet: Sân Mỹ Đình khó tổ chức trận Việt 